# Ingest and View Data

In [1]:
import pandas as pd
from sqlalchemy import create_engine

Creating connection to AWS Postgres database.

In [2]:
POSTGRES_USERNAME = 'root'
POSTGRES_PASSWORD = 'koBREipm4h7CuRKix2Tn'
POSTGRES_HOSTNAME = 'database-1.c3ma8u2guxxs.us-east-2.rds.amazonaws.com'
POSTGRES_PORT = '5432'
POSTGRES_DBNAME = 'postgres'

connection_string = f"postgresql://{POSTGRES_USERNAME}:{POSTGRES_PASSWORD}@{POSTGRES_HOSTNAME}:{POSTGRES_PORT}/{POSTGRES_DBNAME}"
engine = create_engine(connection_string)

In [3]:
query = """
SELECT * FROM darwin
"""

df = pd.read_sql(query, engine)

In [4]:
df.head()

,route_id,unique_id,service_start_date,update_origin,train_platform,working_time_pass,working_time_arrival,working_time_departure,planned_time_arrival,planned_time_departure,...,platform,train_length,estimated_time,source,actual_time,actual_time_class,is_delayed_arrival,is_delayed_departure,source_instance,estimated_time_minutes
0,202407037139232,G39232,2024-07-03,TD,SCROYDN,None,19:31:00,19:31:30,19:31:00,19:31:00,...,4,10.0,None,None,None,None,False,False,None,None
1,202407036785172,C85172,2024-07-03,CIS,LIVSTLL,None,19:37:00,19:38:00,19:37:00,19:38:00,...,A,NaN,None,None,None,None,False,False,None,None
2,202407036735756,C35756,2024-07-03,TD,ESTHRAK,None,19:30:30,19:31:00,19:31:00,19:31:00,...,2,NaN,None,None,None,None,False,False,None,None
3,202407036735756,C35756,2024-07-03,TD,BRMB,None,19:33:00,19:33:30,19:33:00,19:33:00,...,1,NaN,None,None,None,None,False,False,None,None
4,202407036735756,C35756,2024-07-03,TD,BRMBRK,None,19:35:00,19:35:30,19:35:00,19:35:00,...,2,NaN,None,None,None,None,False,False,None,None


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 475574 entries, 0 to 475573
Data columns (total 22 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   route_id                475574 non-null  object 
 1   unique_id               475574 non-null  object 
 2   service_start_date      475574 non-null  object 
 3   update_origin           463381 non-null  object 
 4   train_platform          475574 non-null  object 
 5   working_time_pass       0 non-null       object 
 6   working_time_arrival    475574 non-null  object 
 7   working_time_departure  475574 non-null  object 
 8   planned_time_arrival    436434 non-null  object 
 9   planned_time_departure  434539 non-null  object 
 10  actual_arrival_time     398845 non-null  object 
 11  actual_departure_time   443548 non-null  object 
 12  platform                445648 non-null  object 
 13  train_length            152597 non-null  float64
 14  estimated_time      

# Clean DF 
**Drop Null Columns and Unnecessary Columns**

In [11]:
df_cleaned = df

In [12]:
# check duplicates
duplicates = df_cleaned.duplicated().sum()
print(f'There are {duplicates} duplicates in this dataset.')

There are 15180 duplicates in this dataset.


In [13]:
# to drop duplicates
df_cleaned = df_cleaned.drop_duplicates()

In [14]:
# columns to drop due to null values or used in analysis
columns_to_drop = [
    'working_time_pass',
    'estimated_time',
    'source',
    'actual_time',
    'actual_time_class',
    'source_instance',
    'estimated_time_minutes',
    'train_length', 
    'update_origin',
    'planned_time_arrival',
    'planned_time_departure',
    'platform'
]

df_cleaned = df_cleaned.drop(columns=columns_to_drop)

In [15]:
# check nulls
df_cleaned.isna().sum()

route_id                      0
unique_id                     0
service_start_date            0
train_platform                0
working_time_arrival          0
working_time_departure        0
actual_arrival_time       76436
actual_departure_time     31493
is_delayed_arrival            0
is_delayed_departure          0
dtype: int64

In [19]:
percent_dropped = round( ((1- (df_cleaned.dropna().shape[0]/df_cleaned.shape[0]))*100), 2)
print(f'Dropping the nulls reduces the rows of the dataset by {percent_dropped}%.')

Dropping the nulls reduces the rows of the dataset by 17.15%.


In [ ]:
# to drop remaining nulls
# df_cleaned = df_cleaned.dropna()

# Create Delay DF

In [ ]:
query_delay = "SELECT * FROM darwin WHERE is_delayed_arrival = True AND is_delayed_departure = True"
df_delay = pd.read_sql(query_delay, engine)

In [ ]:
df_delay = df_delay.drop(columns=columns_to_drop)

In [ ]:
df_delay.info()

In [ ]:
df_delay.duplicated().sum()

# Adding Duration Column
Convert times from object datatypes to time datatypes.

In [ ]:
df_cleaned['working_time_arrival'] = pd.to_datetime(df_cleaned['working_time_arrival'], format='%H:%M:%S')
df_cleaned['working_time_departure'] = pd.to_datetime(df_cleaned['working_time_departure'], format='%H:%M:%S')

Complete math to find duration.

In [ ]:
df_cleaned['duration (min)'] = ((df_cleaned['working_time_departure']-df_cleaned['working_time_arrival'])\
                                .dt.total_seconds()/60)

Transition back to time only.

In [ ]:
df_cleaned['working_time_arrival'] = df_cleaned['working_time_arrival'].dt.time
df_cleaned['working_time_departure'] = df_cleaned['working_time_departure'].dt.time

Repeat for Delay Dataframe.

In [ ]:
df_delay['working_time_arrival'] = pd.to_datetime(df_delay['working_time_arrival'], format='%H:%M:%S')
df_delay['working_time_departure'] = pd.to_datetime(df_delay['working_time_departure'], format='%H:%M:%S')
df_delay['duration (min)'] = ((df_delay['working_time_departure']-df_delay['working_time_arrival'])\
                                .dt.total_seconds()/60)
df_delay['working_time_arrival'] = df_delay['working_time_arrival'].dt.time
df_delay['working_time_departure'] = df_delay['working_time_departure'].dt.time

In [ ]:
# Check for zero and negative values
neg_zero_duration_counts = df_cleaned[df_cleaned['duration (min)'] <= 0]['duration (min)'].value_counts().sort_index(ascending=True)

# Display the result
print(neg_zero_duration_counts)

In [ ]:
# Drop the rows with negative values in duration
df_cleaned = df_cleaned[df_cleaned['duration (min)'] > 0]

In [ ]:
# Check
neg_zero_duration_counts = df_cleaned[df_cleaned['duration (min)'] <= 0]['duration (min)'].value_counts().sort_index(ascending=True)

# Display the result
print(neg_zero_duration_counts)

In [ ]:
# Repeating Drop for delay_df to remove negative values in duration

df_delay = df_delay[df_delay['duration (min)'] > 0]

## Adding a Day of the Week Column

In [ ]:
df_cleaned['service_start_date'] = pd.to_datetime(df_cleaned['service_start_date'])
df_cleaned['weekday'] = df_cleaned['service_start_date'].dt.day_name()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
# Top 10 most frequently visited stations
weekday_dist = df_cleaned['weekday'].value_counts()
plt.figure(figsize=(10, 6))
sns.barplot(x=weekday_dist.index, y=weekday_dist.values)
plt.title('Weekday Distribution')
plt.xlabel('Weekday')
plt.ylabel('Number of Data Points')
plt.xticks(rotation=90)
plt.show()

Keeping an eye on this distribtuion to try and keep data even across each weekday.

In [ ]:
df_delay['service_start_date'] = pd.to_datetime(df_delay['service_start_date'])
df_delay['weekday'] = df_delay['service_start_date'].dt.day_name()

# Ingest Rail References Data

In [ ]:
reference_df = pd.read_csv('./raw_data/RailReferences.csv')

## Convert Easting, Northing to Latitude and Longitude

In [ ]:
!pip install pyproj

In [ ]:
from pyproj import Transformer

In [ ]:
#  Define the British National Grid projection (EPSG:27700) and WGS84
osgb_projection = 'epsg:27700'  # British National Grid
wgs84_projection = 'epsg:4326'  # WGS84

In [ ]:
# Initialize the transformer
transformer = Transformer.from_crs(osgb_projection, wgs84_projection)

In [ ]:
# Function to convert British National Grid to latitude and longitude
def convert_osgb_to_latlong(easting, northing):
    longitude, latitude = transformer.transform(easting, northing)
    return longitude, latitude

In [ ]:
# Apply conversion to the DataFrame and create new columns Latitude and Longitude
reference_df[['Latitude', 'Longitude']] = reference_df.apply(lambda row: convert_osgb_to_latlong(row['Easting'], row['Northing']),\
                                         axis=1,\
                                         result_type='expand')

In [ ]:
# Data need for analysis 
data = reference_df[['TiplocCode', 'StationName', 'Latitude', 'Longitude']]
data

### Data Enrichment
Used this section to add identifying information for top Timing Point Locations (*TiplocCode*) not already identified in the supplied "RailReferences.csv" data. 

Research indicates that the Timing Point Location for BONDST, correlates to Bond Street Station$^{1}$ located at 51.514°N, 0.15°W$^{2}$. 

In [ ]:
new_row = pd.DataFrame([['BONDST', 'Bond Street Station', 51.514, -0.15]], columns=data.columns)
data = pd.concat([data, new_row], ignore_index=True)

Research indicates that the Timing Point Location for TOTCTRD, correlates to Tottenham Court Road$^{1}$ located at 51.5207°N, 0.1345°W$^{3}$. 

In [ ]:
new_row = pd.DataFrame([['TOTCTRD', 'Tottenham Court Road', 51.5207, -0.1345]], columns=data.columns)
data = pd.concat([data, new_row], ignore_index=True)

Research indicates that the Timing Point Location for CANWHRF, correlates to Canary Wharf Railway Station$^{1}$ located at 51.5061°N, 0.01578°W$^{4}$. 

In [ ]:
new_row = pd.DataFrame([['CANWHRF', 'Canary Wharf Railway Station', 51.5061, -0.01578]], columns=data.columns)
data = pd.concat([data, new_row], ignore_index=True)

Research indicates that the Timing Point Location for WCHAPXR, correlates to Whitechapel Crossrail$^{1}$ located at 51.5195°N, 0.0612°W$^{5, 6}$. 

In [ ]:
new_row = pd.DataFrame([['WCHAPXR', 'Whitechapel Crossrail', 51.9195, -0.0612]], columns=data.columns)
data = pd.concat([data, new_row], ignore_index=True)

Research indicates that the Timing Point Location for LIVSTLL, correlates to London Liverpool Street Crossrail$^{1}$ located at 51.5186°N, 0.0813°W$^{6, 7}$. 

In [ ]:
new_row = pd.DataFrame([['LIVSTLL', 'London Liverpool Street Crossrail', 51.5186, -0.0813]], columns=data.columns)
data = pd.concat([data, new_row], ignore_index=True)

### Bibliography

1. http://www.railwaycodes.org.uk/crs/crsb.shtm

2. https://en.wikipedia.org/wiki/Bond_Street_station

3. https://en.wikipedia.org/wiki/Tottenham_Court_Road

4. https://en.wikipedia.org/wiki/Canary_Wharf_railway_station

5. https://en.wikipedia.org/wiki/Whitechapel_station

6. https://en.wikipedia.org/wiki/Crossrail

7. https://en.wikipedia.org/wiki/Liverpool_Street_station

## Using Rail References to Replace Train Platform with Station Name

In [ ]:
# Merge df_cleaned with data columns from reference_df

df_merged = df_cleaned.merge(data, left_on='train_platform', right_on='TiplocCode', how='left')

In [ ]:
# checking where train_platform has no related StationName/Tiploccode
nan_rows = df_merged[df_merged[['TiplocCode', 'StationName']].isna().any(axis=1)]

nan_rows[['train_platform', 'TiplocCode', 'StationName']]

In [ ]:
# Identifying most highly frequented unknown Station Names
station_NaN = df_merged[df_merged['StationName'].isna()]['train_platform'].value_counts()
station_NaN.head(20)

In [ ]:
# fill NaN values in the StationName column with the corresponding train_platform values
df_merged['StationName'] = df_merged['StationName'].fillna(df_merged['train_platform'])

# Replace 'Rail Station' with an empty string to make more readable
df_merged['StationName'] = df_merged['StationName'].str.replace(' Rail Station', '')

In [ ]:
df_merged.info()

## Adding a Day of the Wek Column

# Delays

In [ ]:
df_delay = df_delay.merge(data, left_on='train_platform', right_on='TiplocCode', how='left')

In [ ]:
# fill NaN values in the StationName column with the corresponding train_platform values
df_delay['StationName'] = df_delay['StationName'].fillna(df_delay['train_platform'])

# Replace 'Rail Station' with an empty string to make more readable
df_delay['StationName'] = df_delay['StationName'].str.replace(' Rail Station', '')

In [ ]:
df_delay.head()

In [ ]:
df_delay.info()

Determine the minutes delayed.

In [ ]:
df_delay['working_time_arrival'] = pd.to_datetime(df_delay['working_time_arrival'], format='%H:%M:%S')
df_delay['working_time_departure'] = pd.to_datetime(df_delay['working_time_departure'], format='%H:%M:%S')
df_delay['duration (min)'] = ((df_delay['working_time_departure']-df_delay['working_time_arrival'])\
                                .dt.total_seconds()/60)

In [ ]:
df_delay['actual_arrival_time'] = pd.to_datetime(df_delay['actual_arrival_time'], format='%H:%M:%S')
df_delay['actual_departure_time'] = pd.to_datetime(df_delay['actual_departure_time'], format='%H:%M:%S')

In [ ]:
df_delay['depart_delay(min)'] = ((df_delay['actual_departure_time']-df_delay['working_time_departure'])\
                                .dt.total_seconds()/60)

In [ ]:
df_delay['arrive_delay(min)'] = ((df_delay['actual_arrival_time']-df_delay['working_time_arrival'])\
                                .dt.total_seconds()/60)

In [ ]:
df_delay['working_time_arrival'] = df_delay['working_time_arrival'].dt.time
df_delay['working_time_departure'] = df_delay['working_time_departure'].dt.time
df_delay['actual_arrival_time'] = df_delay['actual_arrival_time'].dt.time
df_delay['actual_departure_time'] = df_delay['actual_departure_time'].dt.time

In [ ]:
df_delay.describe()

We can see that the math in the previous step created some erroneous values (minimum of depart_delay(min) is negative.) This will occur in cases where there is no reported actual_departure_time or where the train arrived slightly early. We will get the value zero where the train is not delayed. We will define a function to null delays less than or equal to zero, because these are erroneous on not delays at all. 

In [ ]:
def remove_neg_and_zero(delay):
    if delay <= 0:
        delay = None
    else:
        delay = delay
    return delay

In [ ]:
df_delay['depart_delay(min)'] = df_delay['depart_delay(min)'].apply(remove_neg_and_zero)
df_delay['arrive_delay(min)'] = df_delay['arrive_delay(min)'].apply(remove_neg_and_zero)

In [ ]:
df_delay.describe()

We can see that the are some erroneous high values for depart_delay(min) and arrive_delay(min). We will remove any values over 420minutes (7hrs), assuming those trains are rescheduled rather than delayed.

In [ ]:
def remove_over_420(delay):
    if delay > 420:
        delay = None
    else:
        delay = delay
    return delay

In [ ]:
df_delay['depart_delay(min)'] = df_delay['depart_delay(min)'].apply(remove_over_420)
df_delay['arrive_delay(min)'] = df_delay['arrive_delay(min)'].apply(remove_over_420)

In [ ]:
df_delay.describe()

In [ ]:
df_delay.info()

In [ ]:
# export delay routes parquet
df_delay.to_parquet('./cleaned_data/delay_durations.parquet', index=False)

## Calculate Delay Duration
Take the data where we are able to calucate delays.

In [ ]:
df_delay.head()

In [ ]:
# Create the 'delayed' column
df_merged['delayed'] = df_merged['is_delayed_arrival'] | df_merged['is_delayed_departure']
df_merged['delayed'] = df_merged['delayed'].astype(int)

# Check
df_merged.head()

## Delays  by Route

In [ ]:
# Extract the necessary columns
df_extracted = df_merged[['route_id', 'delayed']]

# Calculate the count of delays for each route_id and TiplocCode
delay_counts_by_route = df_extracted.groupby('route_id')['delayed'].sum().reset_index()
delay_counts_by_route.rename(columns={'delayed': 'delay_counts_by_route'}, inplace=True)

# Calculate the total count of records for each route_id and TiplocCode
total_counts_by_route = df_extracted.groupby('route_id')['route_id'].count().reset_index(name='total_counts_by_route')

# Merge the delay counts and total counts
delay_summary = delay_counts_by_route.merge(total_counts_by_route, on='route_id')

# Calculate the % delayed for each route_id
delay_summary['%_delayed_by_route'] = (delay_summary['delay_counts_by_route'] / delay_summary['total_counts_by_route']) * 100

# Select the relevant columns
delays_by_route = delay_summary[['route_id', 'delay_counts_by_route', 'total_counts_by_route', '%_delayed_by_route']]

# Check
delays_by_route

In [ ]:
# export delay routes parquet
delays_route = delays_by_route.to_parquet('./cleaned_data/delays_by_route.parquet', index=False)

## Delays by Station Name

Changed to aggregate by TiplocCode, because upon further investigation it was found that a Station Name may have various TiplocCodes. Which would inflate the delay counts and station counts for each individual TiplocCode

In [ ]:
df_extracted2 = df_merged[['StationName', 'TiplocCode', 'delayed', 'Latitude', 'Longitude']]

# Calculate the count of delays for each TiplocCode
delay_counts_by_tiploc = df_extracted2.groupby('TiplocCode')['delayed'].sum().reset_index()
delay_counts_by_tiploc.rename(columns={'delayed': 'delay_counts_by_tiploc'}, inplace=True)

# Calculate the total count of records for each TiplocCode
total_counts_by_tiploc = df_extracted2.groupby('TiplocCode')['TiplocCode'].count().reset_index(name='total_counts_by_tiploc')

# Merge the delay counts and total counts
delay_summary_tiploc = delay_counts_by_tiploc.merge(total_counts_by_tiploc, on='TiplocCode')

# Calculate the % delayed for each TiplocCode
delay_summary_tiploc['%_delayed_by_tiploc'] = (delay_summary_tiploc['delay_counts_by_tiploc'] / delay_summary_tiploc['total_counts_by_tiploc']) * 100

# Select the relevant columns
delays_by_tiploc = delay_summary_tiploc[['TiplocCode', 'delay_counts_by_tiploc', 'total_counts_by_tiploc', '%_delayed_by_tiploc']]

# Merge the dataframes on 'TiplocCode' to add latitude, longitude, and StationName
delays_by_tiploc = pd.merge(delays_by_tiploc, df_extracted2[['TiplocCode', 'StationName', 'Latitude', 'Longitude']].drop_duplicates(), on='TiplocCode', how='left')

# Check
delays_by_tiploc[delays_by_tiploc['StationName'] == 'Clapham Junction']

In [ ]:
# exporting delays by station to parquet
delay_stations = delays_by_tiploc.to_csv('./cleaned_data/delays_by_station.csv', index=False, mode="w+")

In [ ]:
# check how is_delayed arrivals/departures compare with delayed column
df_merged['is_delayed_arrival'].value_counts()

In [ ]:
df_merged['is_delayed_departure'].value_counts()

In [ ]:
df_merged['delayed'].value_counts()

Delayed column has same counts as is_delayed_departure. 

# % Delayed by Weekday

Create an aggregated dataframe to show the % of trains delayed by weekday.

In [ ]:
# Extract the necessary columns
df_weekday = df_merged[['weekday', 'delayed']].copy()
df_weekday['count'] = 1
df_weekday = df_weekday.groupby('weekday')[['delayed', 'count']].sum().reset_index()
df_weekday['percent_delayed'] = (df_weekday['delayed'] / df_weekday['count'])*100
df_weekday

In [ ]:
df_weekday.to_csv('./cleaned_data/weekday_data.csv', index=False, mode= "w+")

# Value Counts for Analysis 

In [ ]:
df_merged['service_start_date'].value_counts()

In [ ]:
# most popular routes
df_merged['route_id'].value_counts()

In [ ]:
df_merged['StationName'].value_counts()

# Visuals 

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
# Top 10 most frequently visited stations
top_stations = df_merged['StationName'].value_counts().head(20)
plt.figure(figsize=(10, 6))
sns.barplot(x=top_stations.index, y=top_stations.values)
plt.title('Top 20 Most Frequently Visited Stations')
plt.xlabel('Station')
plt.ylabel('Number of Visits')
plt.xticks(rotation=90)
plt.show()

The chart shows the top 20 most frequently visited stations in the UK rail system, revealing that 13 of these stations are in London. This highlights London's central role in the rail network, driven by its approximate population of 9 million, which constitutes about 13% of the UK's total population. Major hubs like Clapham Junction and London Bridge, as well as key interchange points and commuter stations, contribute to this high traffic, reflecting the substantial demand generated by the city's large population.

Additionally, the chart underscores the importance of regional hubs such as Birmingham New Street, York, and Reading, which serve as central points for their areas, and the significance of rail connections to major airports like Gatwick. The presence of stations like City Thameslink and Whitechapel Crossrail also highlights the integration of various transport modes, providing seamless connectivity for passengers. Overall, London's dominance in the list illustrates its pivotal role in the UK's rail system, driven by its large population and economic activity.

## Distribution of Station Counts 

In [ ]:
df_counts = df_merged['StationName'].value_counts().reset_index()
df_counts.columns = ['StationName', 'Counts']

In [ ]:
# Create the boxplot
plt.figure(figsize=(12, 6))
sns.boxplot(x=df_counts['Counts'])


# Show the plot
plt.show()

In [ ]:
df_counts.describe()

## Duration Distribution 

In [ ]:
df_merged.describe()

In [ ]:
# Histogram for 'duration (min)'
plt.figure(figsize=(10, 6))
plt.hist(df_merged['duration (min)'], bins=100, edgecolor='black')
plt.title('Histogram of Duration (min)')
plt.xlabel('Minutes')
plt.ylabel('Frequency')
plt.show()

Showing true 

In [ ]:
# Histogram for 'duration (min)' with smaller bins and logarithmic x-axis
plt.figure(figsize=(10, 6))
plt.hist(df_merged['duration (min)'], bins=100, edgecolor='black', log=True)

plt.title('Histogram of Duration (min) with Logarithmic Y-axis')
plt.xlabel('Minutes')
plt.ylabel('Frequency')
plt.yscale('log')
plt.show()

This histogram shows the distribution of trip durations from station to station with a logarithmic y-axis. The logarithmic scale is used to make it easier to see and understand data that has a very wide range by squeezing the numbers into a smaller, more readable format. The histogram reveals a high frequency of short trips, with the majority of trip durations clustered around the lower end, particularly between 0 and 50 minutes. There are significant drops in frequency as the duration increases, indicating that long trips are much less common. A few peaks at higher durations suggest some outliers where trips are unusually long.

In [ ]:
# Boxplot for 'duration (min)' with a logarithmic scale
plt.figure(figsize=(10, 6))
plt.boxplot(df_merged['duration (min)'], vert=False, patch_artist=True, showfliers=True)
plt.xscale('log')

# Add mean and median lines
mean_duration = df_merged['duration (min)'].mean()
median_duration = df_merged['duration (min)'].median()
mode_duration = df_merged['duration (min)'].mode().iloc[0]

plt.axvline(mean_duration, color='r', linestyle='--', label=f'Mean: {mean_duration:.2f}')
plt.axvline(median_duration, color='g', linestyle='-', label=f'Median: {median_duration:.2f}')
plt.axvline(mode_duration, color='b', linestyle='-', label=f'Mode: {mode_duration:.2f}')

plt.title('Boxplot of Duration (min) with Logarithmic Scale')
plt.xlabel('Minutes (log scale)')
plt.legend()
plt.show()

This boxplot provides a detailed summary of the distribution, highlighting the mean, median, and mode durations. The median trip duration is 1 minute, with a mean slightly higher at 1.48 minutes, indicating that the distribution is right-skewed with a long tail of longer durations. The mode is at 0.50 minutes, showing that very short trips are most common. The presence of numerous outliers further supports the observation from the histogram that while most trips are short, there are occasional long trips that are significantly longer than the majority. The logarithmic scale on the x-axis helps to spread out the data points, making it easier to see the distribution and identify the concentration of shorter trips and the spread of longer ones.

### Overall Analysis on Trip Duration Distribution. 

The analysis of trip durations reveals that the majority of trips are very short, with most durations clustered between 0 and 50 minutes. This trend makes sense given that the majority of trips occur in densely populated cities like London, where stations are closely spaced, and the demand for frequent, short commutes is high. In such urban environments, the transportation network is designed to handle large volumes of passengers moving over short distances, facilitating quick and efficient travel between closely situated stations.

In cities like London, people often rely on the rail system for daily commutes, errands, and short-distance travel, contributing to the high frequency of brief trips. The shorter trips are reflective of the urban layout, where stations are strategically placed to maximize accessibility and convenience for city dwellers. The presence of numerous outliers with longer trip durations indicates occasional longer journeys, but these are less common. Overall, the data aligns with the expected travel patterns in a densely populated metropolitan area, emphasizing the importance of the rail network in supporting urban mobility.

## Delay Distributions

In [ ]:
plt.figure(figsize=(14, 7))

# Arrival delay histogram
plt.subplot(1, 2, 1)
sns.histplot(df_delay['arrive_delay(min)'], bins=20, color='blue')
plt.title('Histogram of Arrival Delay (min)')
plt.xlabel('Minutes')
plt.ylabel('Frequency')

# Departure delay histogram
plt.subplot(1, 2, 2)
sns.histplot(df_delay['depart_delay(min)'], bins=20, color='green')
plt.title('Histogram of Departure Delay (min)')
plt.xlabel('Minutes')
plt.ylabel('Frequency')

plt.tight_layout()
plt.show()

The histograms display a high frequency of shorter delays, with most clustered between 0 and 50 minutes. As delay time increases, the frequency decreases, indicating that longer delays are less common. This right-skewed distribution shows that while shorter delays are frequent, longer delays, though less common, do occur.

In [ ]:
# Plot boxplot for arrival_delay and depart_delay
plt.figure(figsize=(14, 7))

# Adding mode 
arrival_mode = df_delay['arrive_delay(min)'].mode().iloc[0]
depart_mode = df_delay['depart_delay(min)'].mode().iloc[0]

# Arrival delay boxplot
plt.subplot(1, 2, 1)
sns.boxplot(x=df_delay['arrive_delay(min)'])
plt.title('Boxplot of Arrival Delay (min)')
plt.xlabel('Minutes')
plt.axvline(df_delay['arrive_delay(min)'].mean(), color='red', linestyle='--', label=f'Mean: {df_delay["arrive_delay(min)"].mean():.2f}')
plt.axvline(df_delay['arrive_delay(min)'].median(), color='green', linestyle='-', label=f'Median: {df_delay["arrive_delay(min)"].median():.2f}')
plt.axvline(arrival_mode, color='b', linestyle='-', label=f'Mode: {arrival_mode:.2f}')
plt.legend()

# Departure delay boxplot
plt.subplot(1, 2, 2)
sns.boxplot(x=df_delay['depart_delay(min)'])
plt.title('Boxplot of Departure Delay (min)')
plt.xlabel('Minutes')
plt.axvline(df_delay['depart_delay(min)'].mean(), color='red', linestyle='--', label=f'Mean: {df_delay["depart_delay(min)"].mean():.2f}')
plt.axvline(df_delay['depart_delay(min)'].median(), color='green', linestyle='-', label=f'Median: {df_delay["depart_delay(min)"].median():.2f}')
plt.axvline(depart_mode, color='b', linestyle='-', label=f'Mode: {depart_mode:.2f}')
plt.legend()

plt.tight_layout()
plt.show()

Both boxplots show a substantial interquartile range (IQR), indicating a significant spread in the data. The median values for both arrival and departure delays are around 34.5 minutes and 36 minutes, respectively, suggesting that half of the delays are less than these values, while the other half are greater. The mean values for arrival and departure delays are slightly higher than the median values, at 46.9 minutes and 47.41 minutes, respectively. This discrepancy indicates a right-skewed distribution, with a tail extending towards higher delay times. This right-skewed distribution aligns with the histogram analysis, confirming that while most delays are within a reasonable range, there are instances of significantly longer delays.

**Relating back to the duration distribution**

By comparing these observations with the distribution of trip duration, we can conclude that the UK's rail system experiences a higher frequency of shorter trips and moderate delays, with occasional long durations and severe delays that affect the overall average. This understanding can help in addressing the less frequent but impactful longer trips and delays to improve overall system efficiency.

# Export Desired Dataframe

In [ ]:
output = df_merged.to_parquet('./cleaned_data/UK_Rail_Data.parquet', index=False)

In [ ]:
df_merged.info()

In [ ]:
df_merged.describe()